# Tiny A11y Model Fine-tune Notebook
This notebook fine-tunes DeepSeek-Coder on WCAG + MDN accessibility datasets.

Requirements: Free GPU runtime in Colab (T4 recommended).

In [10]:
# Install required packages
!pip install --upgrade pip
!pip install datasets transformers peft huggingface_hub accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 130.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 62.1 MB/s  0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [datasets]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0

In [3]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()  # Paste your token when prompted

In [4]:
# Fine-tune DeepSeek-Coder with LoRA for accessibility prompts
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, PeftModelForCausalLM
from datasets import load_dataset
import torch


In [5]:
# -----------------------------
# 1️⃣ Load dataset
# -----------------------------
dataset = load_dataset('younglim/a11y-dataset', split='train')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/154k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26 [00:00<?, ? examples/s]

In [6]:
# -----------------------------
# 2️⃣ Load base model and tokenizer
# -----------------------------
base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [7]:
# -----------------------------
# 3️⃣ Add LoRA adapters
# -----------------------------
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj','v_proj'],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, peft_config)
model.train()  # Ensure in train mode


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 2048)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=2048, out_featu

In [8]:
# -----------------------------
# 4️⃣ Tokenize dataset
# -----------------------------
def tokenize_fn(example):
    tokenized = tokenizer(
        example['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
    tokenized['labels'] = tokenized['input_ids'].copy()  # Set labels for causal LM
    return tokenized

tokenized_dataset = dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [11]:
# -----------------------------
# 5️⃣ Training arguments
# -----------------------------
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,   # adjust based on GPU memory
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,                       # mixed precision for faster training
    push_to_hub=True,
    hub_model_id="younglim/tiny-a11y-model",  # your HF repo
)

In [13]:
# -----------------------------
# 6️⃣ Data collator
# -----------------------------
data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True)


In [14]:
# -----------------------------
# 7️⃣ Trainer setup
# -----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-2796754675.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [15]:
# -----------------------------
# 8️⃣ Train
# -----------------------------
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 32021}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: younglim (younglim-gt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=13, training_loss=3.79038326556866, metrics={'train_runtime': 24.1654, 'train_samples_per_second': 1.076, 'train_steps_per_second': 0.538, 'total_flos': 102394666942464.0, 'train_loss': 3.79038326556866, 'epoch': 1.0})

In [17]:
# -----------------------------
# 9️⃣ Save LoRA adapter only
# -----------------------------
model.save_pretrained("tiny-a11y-model")  # LoRA adapter weights

In [18]:
# -----------------------------
# 🔟 Push the fine-tuned LoRA adapter to Hugging Face Hub
# -----------------------------
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...results/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...701581.8f3bd8a2b695.518.0: 100%|##########| 5.52kB / 5.52kB            

  ...701581.8f3bd8a2b695.518.0: 100%|##########| 5.52kB / 5.52kB            

  ...adapter_model.safetensors: 100%|##########| 6.30MB / 6.30MB            

  ...adapter_model.safetensors: 100%|##########| 6.30MB / 6.30MB            

CommitInfo(commit_url='https://huggingface.co/younglim/tiny-a11y-model/commit/a71536885133261b20adc7d165549e1b89e670c0', commit_message='End of training', commit_description='', oid='a71536885133261b20adc7d165549e1b89e670c0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/younglim/tiny-a11y-model', endpoint='https://huggingface.co', repo_type='model', repo_id='younglim/tiny-a11y-model'), pr_revision=None, pr_num=None)

🎉 After running all cells, your fine-tuned Tiny A11y Model will be available at:
[https://huggingface.co/younglim/tiny-a11y-model](https://huggingface.co/younglim/tiny-a11y-model)